### Import Packages

In [3]:
import numpy as np
import scipy
from scipy import optimize
from scipy import signal

### Filtering Algorithm

In [6]:
#Requires data to be entered in the form of an array
#Assumes measurement is over 50ns period, so sampling frequency = length(array)/50ns
def deconvolve(data, energy, alpha, beta, delta, gamma, lpf):
    
    #data - array of discrete time measurements of electronic waveform
    #alpha, beta, delta, gamma - parameters for impulse response function
    #lpf - low pass frequency
    
    totalTime = 50e-9
    sampleSpacing = totalTime/totalSteps
    sampleFreq = 1/sampleSpacing
    
    #Initialize impulse response function
    def impulseResponse(x, alpha, beta, delta, gamma):
        value = (((alpha*x)+beta)**(-delta)-gamma)
        return value
    
    #Initialize impulse response array (values come from function and xAxis)
    xAxis = np.linspace(0, totalTime, totalSteps)
    IR = np.zeros(0)
    for x in range (totalSteps):
        value = impulseResponse(xAxis[x], alpha, beta, delta, gamma)
        if value >= 0:
            IR = np.append(IR, value)
        else:
            IR = np.append(IR, [0])
            
    #Normalize impulse response s.t. integral from -inf to inf = 1
    normConstant = 1/np.sum(IR)
    normIR = normConstant*IR
    
    #Initialize and sort frequency bin
    freqBin = np.fft.fftfreq(totalSteps, sampleSpacing)
    idx1 = np.argsort(freqBin)
    freqBin = freqBin[idx1]
    
    #Create fransfer function (fourier fransform of impulse response)
    TF = np.fft.fft(normIR)
    TF = TF[idx1]
    
    #Take fourier fransform of the data
    fftData = np.fft.fft(data)
    fftData = fftData[idx1]
    
    #Creates filter using transfer function and low pass frequency
    filt = 1/tf
    for x in range (len(filt)):
        if np.abs(freqBin[x])>lpf:
            filt[x] = 0
    
    #Apply filter to data in the fourier domain
    deconvFFT = fftData*filt
    
    #Take Inverse Fourier Transform of filtered data to return to time domain
    deconv = np.abs(np.fft.ifft(deconvFFT))
    
    #Normalize deconvolved signal s.t. the y axis is power
    normConst = energy/np.sum(deconv)
    deconv = normConst*deconv
    
    return deconv